<a href="https://colab.research.google.com/github/GGMmattos/PIC/blob/main/Experimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação das bibliotecas 

In [ ]:
#Importing needed libraries 
!pip install -U scikit-learn

from pandas.core.window.ewm import OnlineExponentialMovingWindow
import pandas as pd #Biblioteca para carregamento de arquivo csv
import numpy as np
import seaborn as sns #Para visualização de gráficos
import matplotlib.pyplot as plt #Para visualização de gráficos
import plotly.express as px #Biblioteca para geração de gráfico dinámico
%matplotlib inline
from collections import Counter


#NLTK tools for text processing 

import re, nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


#Modeling packages 

from sklearn.model_selection import train_test_split
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'


!pip install -U spacy
!python -m spacy download pt
import spacy



# Experimento com a HateBR 1 (sem pré-processamento)



In [29]:
from pandas.core.groupby import base
base_hatebr = pd.read_csv('/content/drive/MyDrive/Dataset/HateBR.csv')
base_hatebr[:5]

,instagram_comments,offensive_language,offensiveness_levels,hate_speech
0,este lixo ...,1,1,-1
1,Mais um lixo,1,1,-1
2,Essa nao tem vergonha na cara!!,1,2,-1
3,Essa mulher é doente.pilantra!,1,3,-1
4,Comunista safada...,1,2,"5,8"


In [30]:
base_hatebr.offensive_language.value_counts()

1    3500
0    3500
Name: offensive_language, dtype: int64

Dividindo em base de teste e base de treinamento



In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    base_hatebr.instagram_comments,
    base_hatebr.offensive_language, 
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = base_hatebr.offensive_language
)

In [32]:
print('Shape of X_train', X_train.shape)
print('Shape of X_test', X_test.shape )

Shape of X_train (5600,)
Shape of X_test (1400,)


In [33]:
y_train.value_counts()

1    2800
0    2800
Name: offensive_language, dtype: int64

In [34]:
y_test.value_counts()

1    700
0    700
Name: offensive_language, dtype: int64

Teste com Naive Bayes




In [45]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


#Criação do objeto Pipeline
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()), #using the ngram_range parameter
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

#Obtem a precisão de X_test e e armazena em y_pred
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

           0       0.85      0.81      0.83       700
           1       0.82      0.86      0.84       700

    accuracy                           0.84      1400
   macro avg       0.84      0.84      0.84      1400
weighted avg       0.84      0.84      0.84      1400



# Experimento com a HateBR 1 (com processamento)



Remoção de Stop Words


In [122]:
# #sem função 

# comments = [] 
# stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

# for words in base_hatebr ['instagram_comments']:
#   only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
#   tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
#   lower_case = [l.lower() for l in tokens] # converter all letters to low case 
#   filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
#   comments.append(' '.join(filtered_result))
  
  # for i in range (len(comments)):
  #   words = nltk.word_tokenize(comments[i])
  #   newwords = [lemmatizer.lemmatize(word) for word in words]
  #   comments[i] = ' '.join(newwords)

In [174]:
npl = spacy.load("pt_core_news_sm")
def preprocess(text):
  doc = npl(text)
  filtered_result = []
  for token in doc:
    if token.is_stop or token.is_punct:
      continue
    filtered_result.append(token.lemma_)
  
  return " ".join(filtered_result)


In [175]:
# print(preprocess(base_hatebr['instagram_comments']))
base_hatebr['preprocessed_comments'] =  base_hatebr['instagram_comments'].apply(preprocess)

In [176]:
base_hatebr

,instagram_comments,offensive_language,offensiveness_levels,hate_speech,preprocessed_comments
0,este lixo ...,1,1,-1,lixo
1,Mais um lixo,1,1,-1,lixo
2,Essa nao tem vergonha na cara!!,1,2,-1,nao vergonha cara
3,Essa mulher é doente.pilantra!,1,3,-1,mulher doente.pilantra
4,Comunista safada...,1,2,"5,8",comunista safar
...,...,...,...,...,...
6995,Time perdendo a credibilidade,0,0,0,time perder credibilidade
6996,Siga em frente Presidente Bolsonaro! Afinal o ...,0,0,0,siga frente Presidente Bolsonaro afinal intere...
6997,Tantas coisas importantes para resolver e fica...,0,0,0,tantas coisa importante resolver ficar dar ênf...
6998,"A TIME escolhe quem eles quiser, isso não sign...",0,0,0,TIME escolhe querer significar


Lematizando com NLTK

In [104]:
v = CountVectorizer(ngram_range=(1,1))
v.fit(comments)
comentarios_vetor = v.transform(comments).toarray()
comentarios_vetor

ValueError: ignored

O TF-IDF mede a importância de uma determinada palavra em relação a um documento e a todo o corpus.

In [ ]:
tfidf = TfidfVectorizer()
transformed_output = tfidf.fit_transform(comments)
print(tfidf.vocabulary_)

In [ ]:
#impressão na ordem
tfidf.get_feature_names_out()

In [ ]:
all_feature_names = tfidf.get_feature_names_out()
for word in all_feature_names:
  index = tfidf.vocabulary_.get(word)
  print(f'{word} {tfidf.idf_[index]}') #A pontuação para termos que aparecem em muitos ducumentos é menor e a pontuação para termos raros é maior.

In [ ]:
comments[:3]

In [ ]:
transformed_output.toarray()[:3]